
# Capstone Project 

# Author : Hamidreza Salahi

# Notebook : 2


<h1>Contents<span class="tocSkip"></span></h1>

<div class="toc"><ul class="toc-item"><li><span><a href="#Importing-Data-" data-toc-modified-id="Importing Data--1"> EDA 


## Importing Data <a class="anchor" id="Importing"></a>

In [1]:
# Importing libraries
import pandas as pd
import numpy as np

In [2]:
# Creating dataframe from dataset zip file
loans = pd.read_csv('C:\\Users\\hamid\\Desktop\\Capstone\\Data\\Loan_status_2007-2020Q3.gzip', compression='gzip', low_memory=False)
loans.head()


KeyboardInterrupt



In [ ]:
loans.tail()

## Creating Sample File <a class="anchor" id="sampling"></a>

Since the original data set is too large, we create a sample data set to work with for now. The sample data set contains about 7% (200000 rows) of the original dataset which are randomly distributed. Duplicated rows are avoided in creating the sample data. 

In [ ]:
# Creating 200000 random indices.
# replace=False ensures that non of the indices are the same --> avoid duplicate rows
rand_index=np.random.choice(loans.index.values, 200000, replace=False)
rand_index

In [75]:
# No duplicated index
pd.DataFrame(rand_index).duplicated().sum()

0

In [76]:
# Using the indices generated above to create the random sample without duplicates
loan_sample = loans.iloc[rand_index].reset_index()

In [77]:
loan_sample.head()

,index,Unnamed: 0,id,loan_amnt,funded_amnt,funded_amnt_inv,term,int_rate,installment,grade,...,hardship_start_date,hardship_end_date,payment_plan_start_date,hardship_length,hardship_dpd,hardship_loan_status,orig_projected_additional_accrued_interest,hardship_payoff_balance_amount,hardship_last_payment_amount,debt_settlement_flag
0,435359,81941,121350021,22000.0,22000.0,22000.0,60 months,10.42%,472.00,B,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N
1,1954886,87125,63287670,11000.0,11000.0,11000.0,60 months,19.19%,286.50,E,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N
2,2438264,15521,81466670,26500.0,26500.0,26500.0,60 months,12.99%,602.83,C,...,NaN,NaN,NaN,NaN,NaN,NaN,249.55803,7740.08,602.83,N
3,133632,91096,5524700,10000.0,10000.0,10000.0,36 months,10.16%,323.43,B,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N
4,2733364,10101,102433652,20000.0,20000.0,20000.0,36 months,11.49%,659.43,B,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Y


In [78]:
#Dropping the generated columns which are useless
loan_sample.drop(["index","Unnamed: 0" , "id"] , axis = 1 , inplace=True)

## Data Cleaning <a class="anchor" id="cleaning"></a>

At a glance, it is seen that the data set has a lot of Nan values, specially in some certain columns. We will drop the columns which have more than 10% of their data missing

### Dropping NaN columns

In [83]:
loan_sample.isna().sum()

loan_amnt                        1
funded_amnt                      1
funded_amnt_inv                  1
term                             1
int_rate                         1
                              ... 
total_bal_ex_mort             3414
total_bc_limit                3414
total_il_high_credit_limit    4747
hardship_flag                 2618
debt_settlement_flag             1
Length: 90, dtype: int64

In [80]:
# Finding all columns with more than 10% values missing
nan_cols = [i for i in loan_sample.columns if loan_sample[i].isnull().sum() > 0.1*len(loan_sample)]

In [92]:
# Dropping nan columns 
loan_sample.drop(nan_cols , axis=1, inplace=True)
loan_sample.shape

KeyError: "['mths_since_last_delinq', 'mths_since_last_record', 'next_pymnt_d', 'mths_since_last_major_derog', 'annual_inc_joint', 'dti_joint', 'verification_status_joint', 'open_acc_6m', 'open_act_il', 'open_il_12m', 'open_il_24m', 'mths_since_rcnt_il', 'total_bal_il', 'il_util', 'open_rv_12m', 'open_rv_24m', 'max_bal_bc', 'all_util', 'inq_fi', 'total_cu_tl', 'inq_last_12m', 'mths_since_recent_bc_dlq', 'mths_since_recent_inq', 'mths_since_recent_revol_delinq', 'revol_bal_joint', 'sec_app_fico_range_low', 'sec_app_fico_range_high', 'sec_app_earliest_cr_line', 'sec_app_inq_last_6mths', 'sec_app_mort_acc', 'sec_app_open_acc', 'sec_app_revol_util', 'sec_app_open_act_il', 'sec_app_num_rev_accts', 'sec_app_chargeoff_within_12_mths', 'sec_app_collections_12_mths_ex_med', 'hardship_type', 'hardship_reason', 'hardship_status', 'deferral_term', 'hardship_amount', 'hardship_start_date', 'hardship_end_date', 'payment_plan_start_date', 'hardship_length', 'hardship_dpd', 'hardship_loan_status', 'or

In [93]:
data_Dic.head()

,LoanStatNew,Description
0,acc_now_delinq,The number of accounts on which the borrower i...
1,acc_open_past_24mths,Number of trades opened in past 24 months.
2,addr_state,The state provided by the borrower in the loan...
3,all_util,Balance to credit limit on all trades
4,annual_inc,The self-reported annual income provided by th...


In [94]:
# Joining the dictionary and column names
desc_loans_colms = pd.merge(pd.DataFrame({"col_name": list(loan_sample.columns)}), data_Dic, \
               how='inner', right_on='LoanStatNew', left_on='col_name')
desc_loans_colms.drop(['col_name'], axis =1, inplace=True)

In [95]:
# Displaying the dictionary defining each column
from IPython.display import display
with pd.option_context('display.max_rows', 90, 'display.max_columns', 3 , 'display.max_colwidth' , -1):
    display(desc_loans_colms)

,LoanStatNew,Description
0,loan_amnt,"The listed amount of the loan applied for by the borrower. If at some point in time, the credit department reduces the loan amount, then it will be reflected in this value."
1,funded_amnt,The total amount committed to that loan at that point in time.
2,funded_amnt_inv,The total amount committed by investors for that loan at that point in time.
3,term,The number of payments on the loan. Values are in months and can be either 36 or 60.
4,int_rate,Interest Rate on the loan
5,installment,The monthly payment owed by the borrower if the loan originates.
6,grade,LC assigned loan grade
7,sub_grade,LC assigned loan subgrade
8,emp_title,The job title supplied by the Borrower when applying for the loan.*
9,emp_length,Employment length in years. Possible values are between 0 and 10 where 0 means less than one year and 10 means ten or more years.


In [99]:
loan_sample.columns.shape

(90,)

* There are 90 columns in the dataset. However, looking at the data dictionary, one sees only 89 columns. The description of the column `total_rev_hi_lim` is missing in the Data Dictionary

<font size="3.5"> There are still 90 columns left even after dropping NaN columns! At this point, it is important to keep only the relevant columns to avoid confussion in the models which are going to be built on the sample data. It is assumed that keeping the following columns suffices to answer the bussiness question asked here.
</font>

* loan_amnt
* term
* int_rate
* emp_title
* emp_length
* home_ownership
* annual_inc
* verification_status
* <font size="4.5">loan_status</font>
* purpose
* addr_state
* dti
* delinq_2yrs
* fico_range_low
* 

In [131]:
loan_sample['pub_rec_bankruptcies'].corr(loan_sample['loan_amnt'])

-0.08182354495364293

In [16]:
loan_sample.to_csv('C:\\Users\\hamid\\Desktop\\Capstone\\Data\\loan_sample.csv' , index=False)

In [ ]:
loan_sample = pd.read_csv('C:\\Users\\hamid\\Desktop\\Capstone\\Data\\loan_sample.csv')

loan_sample.columns